# Yolo3 Finetuning with AWS

This series of notebooks demonstrates how to finetune pretrained YOLO v3 (aka YOLO3) using MXNet on AWS.

**This notebook** walks through using the [SageMaker Hyperparameter Tuning Job](https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning-how-it-works.html) tool to finding optmized hypterparameter and finetune the model.

**Follow-on** the content of the notebooks shows:

* How to use MXNet YOLO3 pretrained model
* How to create Ground-Truth dataset from images the model mis-detected
* How to finetune the model using the created dataset
* Load your finetuned model and Deploy Sagemaker-Endpoint with it.
* Apply Elastic Inference to your endpoint.

## Pre-requisites

This notebook is designed to be run in Amazon SageMaker. To run it (and understand what's going on), you'll need:

* Basic familiarity with Python, [MXNet](https://mxnet.apache.org/), [AWS S3](https://docs.aws.amazon.com/s3/index.html), [Amazon Sagemaker](https://aws.amazon.com/sagemaker/)
* To create an **S3 bucket** in the same region, and ensure the SageMaker notebook's role has access to this bucket.
* Sufficient [SageMaker quota limits](https://docs.aws.amazon.com/general/latest/gr/aws_service_limits.html#limits_sagemaker) set on your account to run GPU-accelerated spot training jobs.

## Cost and runtime

Depending on your configuration, this demo may consume resources outside of the free tier but should not generally be expensive because we'll be training on a small number of images. You might wish to review the following for your region:

* [Amazon SageMaker pricing](https://aws.amazon.com/sagemaker/pricing/)

The standard `ml.t2.medium` instance should be sufficient to run the notebooks.

We will use GPU-accelerated instance types for training and hyperparameter optimization, and use spot instances where appropriate to optimize these costs.

As noted in the step-by-step guidance, you should take particular care to delete any created SageMaker real-time prediction endpoints when finishing the demo.

# Step 0: Dependencies and configuration

As usual we'll start by loading libraries, defining configuration, and connecting to the AWS SDKs:

In [7]:
%load_ext autoreload
%autoreload 1

# Built-Ins:
import os
import json
from datetime import datetime
from glob import glob
from pprint import pprint
from base64 import b64encode, b64decode
from matplotlib import pyplot as plt

# External Dependencies:
import boto3
import sagemaker
import numpy as np
from sagemaker.mxnet import MXNet
from gluoncv.utils import download, viz
from botocore.exceptions import ClientError

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
%store -r

In [23]:
iam = boto3.client('iam')
role = iam.get_role(RoleName=role_name)['Role']['Arn']

## Step 1: Get best job informations 

In [12]:
analytics = sagemaker.HyperparameterTuningJobAnalytics(training_job_name)
df = analytics.dataframe().sort_values('FinalObjectiveValue')
df[:3]

,lr,momentum,optimizer,wd,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
9,0.000218,0.885406,"""adam""",0.967501,yolo-htj-batch-0-200316-1644-015-2493e60c,Completed,7.664004,2020-03-16 17:18:30+09:00,2020-03-16 17:23:09+09:00,279.0
6,0.000349,0.735787,"""sgd""",0.981868,yolo-htj-batch-0-200316-1644-018-0d72777e,Completed,7.828395,2020-03-16 17:26:49+09:00,2020-03-16 17:31:11+09:00,262.0
4,0.000132,0.976250,"""sgd""",0.208593,yolo-htj-batch-0-200316-1644-020-c4d5fd9f,Completed,8.670670,2020-03-16 17:33:35+09:00,2020-03-16 17:37:52+09:00,257.0


In [16]:
best_job_name = df.iloc[0]['TrainingJobName']
print(best_job_name)
%store best_job_name

yolo-htj-batch-0-200316-1644-015-2493e60c
Stored 'best_job_name' (str)


## Step 2: Create Sagemaker Model

In [20]:
model_name = 'yolo-model-0'
%store model_name

Stored 'model_name' (str)


In [18]:
model_output_path

's3://sagemaker-ap-northeast-2-929831892372/models'

In [24]:
mxnet_model = sagemaker.mxnet.model.MXNetModel(
    name=model_name,
    model_data=f'{model_output_path}/{best_job_name}/output/model.tar.gz', 
    role=role, 
    entry_point='inference_cpu.py',
    source_dir='src',
    framework_version='1.4.1',
    py_version='py3',
)

## Step 3: Deploy Model

In [ ]:
predictor = mxnet_model.deploy(
    instance_type='ml.c5.xlarge', initial_instance_count=1
)

## Step 4: Invoke Sagemaker Endpoint

In [8]:
bimage = None
download('https://sportshub.cbsistatic.com/i/r/2019/11/15/10869f78-1378-4aa5-b36b-085607ae3387/thumbnail/770x433/f3276ac966a56b7cb45987869098cddb/lionel-messi-argentina-brazil.jpg', path='soccer.jpg')
with open('soccer.jpg', 'rb') as fp:
    bimage = fp.read()
s = b64encode(bimage).decode('utf-8')

In [ ]:
%%time
res = predictor.predict({
    'short': 416,
    'image': s
})
print(res['shape'])

In [ ]:
ax = viz.plot_bbox(mx.image.imresize(mx.image.imdecode(bimage), 492, 416), mx.nd.array(res['bbox']), mx.nd.array(res['score']), mx.nd.array(res['cid']), class_names=['person'])

## Step5: Invoke custom Endpoint (Optional)

get predictor by name and invoke endpoint

In [2]:
def serializer(data): 
    return json.dumps(data).encode('utf-8')

def deserializer(body, content_type):
    return json.loads(body.read().decode('utf-8'))

In [ ]:
predictor = sagemaker.predictor.RealTimePredictor(
    endpoint='yolo-model-0',
    content_type='application/json', 
    accept='application/json',
    serializer=serializer,
    deserializer=deserializer,
)

In [ ]:
%%time
res = predictor.predict({
    'short': 416,
    'image': s
})
print(res['shape'])

In [ ]:
ax = viz.plot_bbox(mx.image.imresize(mx.image.imdecode(bimage), 492, 416), mx.nd.array(res['bbox']), mx.nd.array(res['score']), mx.nd.array(res['cid']), class_names=['person'])